In [2]:
    import pandas as pd
    from sklearn.model_selection import train_test_split
    import seaborn as sns
    import matplotlib.pyplot as plt
    import numpy as np

    from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
    from sklearn.compose import ColumnTransformer
    from sklearn.pipeline import Pipeline
    from sklearn.linear_model import LinearRegression, RidgeClassifier
    from xgboost import XGBRegressor, XGBClassifier
    from sklearn.model_selection import KFold 
    from sklearn.model_selection import cross_val_score
    from sklearn.metrics import roc_auc_score, accuracy_score, roc_curve
    import lightgbm as lgb
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
    from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
df = pd.read_csv('/content/drive/MyDrive/Nuclio/Capstone/Copia de BTCUSDT_velas_15.csv', sep=',', index_col=0)
#df.drop(['Volume','Number of trades','Quote assert volume',	'Taker buy base asset volume',	'Taker buy quote asset volume',	'Ignore'],inplace=True, axis=1)
df = df[['Open',	'High',	'Low', 'Close']]
df.drop(index = df.index[-1],inplace=True, axis=1)
df

,Open,High,Low,Close
0,3701.23,3703.72,3689.88,3697.31
1,3697.91,3698.75,3690.56,3694.20
2,3693.96,3705.00,3693.00,3702.39
3,3702.05,3713.00,3695.02,3700.31
4,3700.20,3702.73,3691.22,3701.03
...,...,...,...,...
138834,16398.72,16420.27,16361.75,16416.82
138835,16416.82,16459.18,16373.43,16382.34
138836,16382.34,16428.99,16373.14,16423.25
138837,16423.25,16442.35,16410.25,16440.90


In [5]:
d = df.copy()

In [6]:
df_predicciones = pd.read_csv('/content/drive/MyDrive/Nuclio/Capstone/Copia de BTC_15_df_extractoPredicciones', sep=',', index_col=0)
#df_predicciones.drop(['Volume','Number of trades','Quote assert volume',	'Taker buy base asset volume',	'Taker buy quote asset volume',	'Ignore'],inplace=True, axis=1)
df_predicciones = df_predicciones[['Open',	'High',	'Low', 'Close']]
df_predicciones

,Open,High,Low,Close
Close_Time,,,,
2022-12-20 00:14:59.999,16438.880000,16456.000000,16397.200000,16411.280000
2022-12-20 00:29:59,16411.280000,16664.988281,16495.917969,16601.783203
2022-12-20 00:44:59,16601.783203,16652.078125,16616.812500,16600.232422
2022-12-20 00:59:59,16600.232422,16658.816406,16619.580078,16633.375000
2022-12-20 01:14:59,16633.375000,16661.388672,16618.949219,16630.179688
...,...,...,...,...
2023-01-04 17:59:59,17566.755859,17619.230469,17453.578125,17565.769531
2023-01-04 18:14:59,17565.769531,17624.572266,17436.710938,17557.808594
2023-01-04 18:29:59,17557.808594,17626.128906,17453.591797,17551.328125


In [7]:
lista_vacia = []
for i in range(0,1516):
    df = df.append(df_predicciones.iloc[i])

    df['diff'] = df['Close'] - df['Close'].shift(1)
    df.loc[df['diff'] >= 0, 'Up_DownM'] = 1
    df.loc[df['diff'] < 0, 'Up_DownM'] = 0  
    df['Up_DownM'] = df['Up_DownM'].shift(-1)   
    del df['diff']        
    df_T = df.iloc[-1]
    df.drop(index=df.index[-1],inplace=True, axis=0)

    target = 'Up_DownM'
    features = [i for i in df.columns if i != target]

    y = df[target]
    X = df[features]

    X_train, X_test, y_train, y_test = train_test_split(
                                            X, 
                                            y, 
                                            test_size = 0.3, # tamaño del split aleatorio 
                                            # aunque el split sea aleatorio, stratify garantiza que el ratio de exitos sea el mismo en train y test
                                            random_state = 3 # Guarda el split en el seed 42 por si volvemos a ejecutar la celda, sea el mismo split
                                        )
    
    transform_pipe = ColumnTransformer(transformers = [
        ("scaler", MinMaxScaler(), [0,1,2,3]),
        # ('encoder', OneHotEncoder(), [4])
    
        
      ])

    pipe = Pipeline(steps = [
        ("transform", transform_pipe),
        
      ])


    X_train = pipe.fit_transform(X_train)
    X_test = pipe.transform(X_test)

    C = lgb.LGBMClassifier()
    model = C.fit(X_train, y_train) 

    #Predict

    df_T = pd.DataFrame(df_T)
    df_T = df_T.T


    target = 'Up_DownM'
    features = [i for i in df.columns if i != target]

    y_pred = df_T[target]
    X_pred = df_T[features]

    X_pred = pipe.transform(X_pred)
    y_pred = model.predict(X_pred)

    df_T['Up_DownM'] = y_pred
    
    #DataFrame

    df = df.append(df_T)
    df_T['Up_DownM'] = y_pred
    # df.reset_index(inplace=True)
    # del df['index']
    print('-------------------------------------')
    print(f'Intento: {i}')
    print(f'Predicción {y_pred}')
    if i == 1515:
      # df.to_csv()
      lista_vacia.append(df)
      df.to_csv('/content/drive/MyDrive/Nuclio/Capstone/BTC_15_Modelo_Up_DownM_Predicho.csv')
      
    else:
      continue

-------------------------------------
Intento: 0
Predicción [1.]
-------------------------------------
Intento: 1
Predicción [1.]
-------------------------------------
Intento: 2
Predicción [1.]
-------------------------------------
Intento: 3
Predicción [1.]
-------------------------------------
Intento: 4
Predicción [1.]
-------------------------------------
Intento: 5
Predicción [1.]
-------------------------------------
Intento: 6
Predicción [1.]
-------------------------------------
Intento: 7
Predicción [1.]
-------------------------------------
Intento: 8
Predicción [0.]
-------------------------------------
Intento: 9
Predicción [1.]
-------------------------------------
Intento: 10
Predicción [1.]
-------------------------------------
Intento: 11
Predicción [1.]
-------------------------------------
Intento: 12
Predicción [1.]
-------------------------------------
Intento: 13
Predicción [1.]
-------------------------------------
Intento: 14
Predicción [1.]
--------------------

In [8]:
df_predicciones.head()

,Open,High,Low,Close
Close_Time,,,,
2022-12-20 00:14:59.999,16438.880000,16456.000000,16397.200000,16411.280000
2022-12-20 00:29:59,16411.280000,16664.988281,16495.917969,16601.783203
2022-12-20 00:44:59,16601.783203,16652.078125,16616.812500,16600.232422
2022-12-20 00:59:59,16600.232422,16658.816406,16619.580078,16633.375000
2022-12-20 01:14:59,16633.375000,16661.388672,16618.949219,16630.179688
